In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score, classification_report
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from transformers import AdamW
from tqdm import tqdm

In [ ]:
pip install fugashi ipadic

In [ ]:
language="jp"
modelname="JapaneseBERT"
input = "gpt4"
test = "OPENMWE"
filtered= True

In [ ]:
if input == "gpt4":
  gpt4_flag=True
else:
  gpt4_flag=False

In [ ]:
# JP encoding='euc-jp' other encoding normal CHECK ALWAYS DATASETS
df_train = pd.read_csv(f'train_gpt4_jp_zero_shot.csv',encoding='euc-jp')
df_eval = pd.read_csv(f'eval_gpt4_jp_zero_shot.csv',encoding='euc-jp')
df_test = pd.read_csv(f'jp_test.csv',encoding='euc-jp')

In [ ]:
def get_category_mapping(language):
    if language in ['en', 'jp']:
        category_mapping = {
            'figurative': 'idiom',
            'literal': 'nonidiom'
        }
    elif language == 'tr':
        category_mapping = {
            'mecaz': 'idiom',
            'gerçek': 'nonidiom'
        }
    elif language == 'it':
        category_mapping = {
            'figurato': 'idiom',
            'letterale': 'nonidiom'
        }
    else:
        raise ValueError(f"Language '{language}' is not supported.")

    return category_mapping

In [ ]:
category_mapping = get_category_mapping(language)
print(category_mapping)

{'figurative': 'idiom', 'literal': 'nonidiom'}


In [ ]:
if gpt4_flag or language in ['en', 'jp']:
  df_train['category'] = df_train['category'].map(category_mapping)
df_train

,submission,category,idiom,type
0,彼の誠実な対応にはいつも頭が下がる。,idiom,頭が下がる,Zero-shot
1,長年悪事に手を染めていたが、彼は足を洗って更生を決意した。,idiom,足を洗う,Zero-shot
2,病気で苦しむ友人を見舞うたび、胸が痛む。,idiom,胸が痛む,Zero-shot
3,熱しやすく冷めやすい社長のもとで働くことは、常に新たな挑戦に直面することを意味する。,idiom,熱しやすく冷めやすい,Zero-shot
4,彼はこの小説が彼の作家生活の集大成と考え、筆を置いて物語に幕を閉じた。,idiom,幕を閉じる,Zero-shot
...,...,...,...,...
4507,セメントの彫刻は何十年もの間外に置かれていたため、元々の色を失った。,nonidiom,色を失う,Zero-shot
4508,彼は初めてサーフィンに挑戦し、少しの間だけでしたが、成功して波に乗る喜びを味わった。,nonidiom,波に乗る,Zero-shot
4509,金属を研磨するときは、火花が散らないように安全対策を行います。,nonidiom,火花を散らす,Zero-shot
4510,冷えた部屋で体を温めるために、暖房器具の熱を少し上げました。,nonidiom,熱を上げる,Zero-shot


In [ ]:
if gpt4_flag or language in ['en', 'jp']:
  df_eval['category'] = df_eval['category'].map(category_mapping)
df_eval

,submission,category,idiom,type
0,この学校で教師として尽力してきたから、ここに骨を埋めるのが相応しい。,idiom,骨を埋める,Zero-shot
1,この広い庭の手入れは、季節ごとに骨が折れる作業が絶えない。,idiom,骨が折れる,Zero-shot
2,社内の二人の組み合わせは水と油のごとく、一緒に仕事をしてもなかなか成果が出ない。,idiom,水と油,Zero-shot
3,複雑なプログラムのバグを探し出す作業は、試行錯誤が多く骨が折れる。,idiom,骨が折れる,Zero-shot
4,彼は新しい事業計画が予算オーバーになるかもしれないと考え、頭を痛めている。,idiom,頭を痛める,Zero-shot
...,...,...,...,...
1123,彼女はハイキング中に拾った様々な種類の実を布で包み、それを木に結び付けてマーカーとして使った。,nonidiom,実を結ぶ,Zero-shot
1124,展示施設のリニューアルオープンで、最新の展示空間を隠していた幕が開かれ、訪れた人々が驚嘆した。,nonidiom,幕が開く,Zero-shot
1125,子供達が浮き輪を使って波に乗るのを見るのは、いつも楽しい光景です。,nonidiom,波に乗る,Zero-shot
1126,散歩後、彼は足を洗いながら手も一緒に洗った。,nonidiom,足を洗う,Zero-shot


In [ ]:
if language in ['en', 'jp']:
  df_test['category'] = df_test['category'].map(category_mapping)

In [ ]:
df_test

,category,id,idiom,submission
0,idiom,35,あぐらをかく,この財政赤字の時代に民間だけが運営・税務対策・価格競争を生き抜いて戦っている環境の中で、何も...
1,idiom,35,あぐらをかく,教育改革云々という言葉が連日、新聞紙面やテレビを賑わしていますが、本校も百年近い歴史に<あぐ...
2,idiom,35,あぐらをかく,ソフトウェアの利用者は、それらの工夫の為に、そのソフトウェアが問題を抱えていると自覚する機会...
3,idiom,35,あぐらをかく,自然を大切に暮らしてきた人たちの心の暖かさや、決して伝統の上に<あぐらを><かくことなく>歴...
4,idiom,35,あぐらをかく,明治１６年創業の同店は、「のれん」に<あぐらを><かくことなく、>きものが好きな方々に少しで...
...,...,...,...,...
15234,nonidiom,2621,鼻を鳴らす,「なぜ、この子を殴る！？ 今しがたの件に、何の関係があったというのだ！！」 男はフンと<鼻を...
15235,nonidiom,2621,鼻を鳴らす,「ガルア、もういいわ……離しなさい……」 リリスとシェシェルに少し離れて桃園市の上空を飛んで...
15236,nonidiom,2621,鼻を鳴らす,ぐす、と<鼻を><鳴らす>敏生に、森はこの少年に感じさせてしまった孤独感に胸が苦しくなった。
15237,nonidiom,2621,鼻を鳴らす,ある時には，ハミングするような音を出したり，<鼻を>クンクン<鳴らしたり，また，>ある時には...


In [ ]:
if filtered == True:
  df_train = df_train.groupby(['idiom', 'category']).apply(lambda x: x.sample(n=32, replace=False)).reset_index(drop=True)
  df_eval = df_eval.groupby(['idiom', 'category']).apply(lambda x: x.sample(n=8, replace=False)).reset_index(drop=True)

<ipython-input-112-5d2163afaadf>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_train = df_train.groupby(['idiom', 'category']).apply(lambda x: x.sample(n=32, replace=False)).reset_index(drop=True)
<ipython-input-112-5d2163afaadf>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_eval = df_eval.groupby(['idiom', 'category']).apply(lambda x: x.sample(n=8, replace=False)).reset_index(drop=True)


In [ ]:
df_train['category'] = df_train['category'].str.lower()
df_eval['category'] = df_eval['category'].str.lower()
df_test['category'] = df_test['category'].str.lower()

In [ ]:
df_train.head()

,submission,category,idiom,type
0,多くの賞を受賞した作家も、あぐらをかくことなく新たな作品を書き続けている。,idiom,あぐらをかく,Zero-shot
1,売り上げが過去最高を記録した今月も、スタッフ全員であぐらをかかずに次なる目標に取り組んでいる。,idiom,あぐらをかく,Zero-shot
2,チーム全員が成績を良くして一息つき、少しあぐらをかいてしまったが、すぐに気を引き締める必要がある。,idiom,あぐらをかく,Zero-shot
3,成績優秀な彼だが、あぐらをかいて勉強を怠ることなく、常に自己研鑽に励んでいる。,idiom,あぐらをかく,Zero-shot
4,市場での成功により競合他社が油断している間に、我々はあぐらをかかずに新たな戦略を練っている。,idiom,あぐらをかく,Zero-shot


In [ ]:
df_eval.head()

,submission,category,idiom,type
0,競争が激化している業界において、成果を上げたからといってあぐらをかくのは危険だ。,idiom,あぐらをかく,Zero-shot
1,彼女は昇進して課長になったが、あぐらをかくことなく部下の指導にあたっている。,idiom,あぐらをかく,Zero-shot
2,一度の大きな成果で満足することなく、あぐらをかかずに更なるチャレンジに挑んでいる。,idiom,あぐらをかく,Zero-shot
3,長年の努力の後、彼はついに業界のリーダーになったが、あぐらをかかずにさらに技術革新に力を注い...,idiom,あぐらをかく,Zero-shot
4,私は初めての大会で好成績を収めたが、それに満足してあぐらをかくわけにはいかない。,idiom,あぐらをかく,Zero-shot


In [ ]:
df_test.head()

,category,id,idiom,submission
0,idiom,35,あぐらをかく,この財政赤字の時代に民間だけが運営・税務対策・価格競争を生き抜いて戦っている環境の中で、何も...
1,idiom,35,あぐらをかく,教育改革云々という言葉が連日、新聞紙面やテレビを賑わしていますが、本校も百年近い歴史に<あぐ...
2,idiom,35,あぐらをかく,ソフトウェアの利用者は、それらの工夫の為に、そのソフトウェアが問題を抱えていると自覚する機会...
3,idiom,35,あぐらをかく,自然を大切に暮らしてきた人たちの心の暖かさや、決して伝統の上に<あぐらを><かくことなく>歴...
4,idiom,35,あぐらをかく,明治１６年創業の同店は、「のれん」に<あぐらを><かくことなく、>きものが好きな方々に少しで...


In [ ]:
# Encode the 'category' column
label_encoder = LabelEncoder()
df_train['category_encoded'] = label_encoder.fit_transform(df_train['category'])
df_eval['category_encoded'] = label_encoder.transform(df_eval['category'])
df_test['category_encoded'] = label_encoder.transform(df_test['category'])

In [ ]:
df_train

,submission,category,idiom,type,category_encoded
0,多くの賞を受賞した作家も、あぐらをかくことなく新たな作品を書き続けている。,idiom,あぐらをかく,Zero-shot,0
1,売り上げが過去最高を記録した今月も、スタッフ全員であぐらをかかずに次なる目標に取り組んでいる。,idiom,あぐらをかく,Zero-shot,0
2,チーム全員が成績を良くして一息つき、少しあぐらをかいてしまったが、すぐに気を引き締める必要がある。,idiom,あぐらをかく,Zero-shot,0
3,成績優秀な彼だが、あぐらをかいて勉強を怠ることなく、常に自己研鑽に励んでいる。,idiom,あぐらをかく,Zero-shot,0
4,市場での成功により競合他社が油断している間に、我々はあぐらをかかずに新たな戦略を練っている。,idiom,あぐらをかく,Zero-shot,0
...,...,...,...,...,...
3003,彼女は本を読み終え、満足感で一息ついた時に鼻を鳴らした。,nonidiom,鼻を鳴らす,Zero-shot,1
3004,深い瞑想の中で、彼は静かに息を吸い、鼻を鳴らしながらゆっくり息を吐いた。,nonidiom,鼻を鳴らす,Zero-shot,1
3005,彼女は集中して読書をしながら、不意に鳴らす鼻音が静かな部屋に響いた。,nonidiom,鼻を鳴らす,Zero-shot,1
3006,緊張を解消しようと、彼は深呼吸をしてから鼻を鳴らす癖がある。,nonidiom,鼻を鳴らす,Zero-shot,1


In [ ]:
df_eval

,submission,category,idiom,type,category_encoded
0,競争が激化している業界において、成果を上げたからといってあぐらをかくのは危険だ。,idiom,あぐらをかく,Zero-shot,0
1,彼女は昇進して課長になったが、あぐらをかくことなく部下の指導にあたっている。,idiom,あぐらをかく,Zero-shot,0
2,一度の大きな成果で満足することなく、あぐらをかかずに更なるチャレンジに挑んでいる。,idiom,あぐらをかく,Zero-shot,0
3,長年の努力の後、彼はついに業界のリーダーになったが、あぐらをかかずにさらに技術革新に力を注い...,idiom,あぐらをかく,Zero-shot,0
4,私は初めての大会で好成績を収めたが、それに満足してあぐらをかくわけにはいかない。,idiom,あぐらをかく,Zero-shot,0
...,...,...,...,...,...
747,体調が悪いときによく鼻を鳴らす習慣が彼にはある。,nonidiom,鼻を鳴らす,Zero-shot,1
748,運動後、彼は鼻を鳴らしながらベンチに座って休息を取った。,nonidiom,鼻を鳴らす,Zero-shot,1
749,犬が遊び疲れて横になり、安心感からか大きく鼻を鳴らして眠り込んだ。,nonidiom,鼻を鳴らす,Zero-shot,1
750,疲れてベッドに横になった彼は、うとうとしながら時々鼻を鳴らしている。,nonidiom,鼻を鳴らす,Zero-shot,1


In [ ]:
df_test

,category,id,idiom,submission,category_encoded
0,idiom,35,あぐらをかく,この財政赤字の時代に民間だけが運営・税務対策・価格競争を生き抜いて戦っている環境の中で、何も...,0
1,idiom,35,あぐらをかく,教育改革云々という言葉が連日、新聞紙面やテレビを賑わしていますが、本校も百年近い歴史に<あぐ...,0
2,idiom,35,あぐらをかく,ソフトウェアの利用者は、それらの工夫の為に、そのソフトウェアが問題を抱えていると自覚する機会...,0
3,idiom,35,あぐらをかく,自然を大切に暮らしてきた人たちの心の暖かさや、決して伝統の上に<あぐらを><かくことなく>歴...,0
4,idiom,35,あぐらをかく,明治１６年創業の同店は、「のれん」に<あぐらを><かくことなく、>きものが好きな方々に少しで...,0
...,...,...,...,...,...
15234,nonidiom,2621,鼻を鳴らす,「なぜ、この子を殴る！？ 今しがたの件に、何の関係があったというのだ！！」 男はフンと<鼻を...,1
15235,nonidiom,2621,鼻を鳴らす,「ガルア、もういいわ……離しなさい……」 リリスとシェシェルに少し離れて桃園市の上空を飛んで...,1
15236,nonidiom,2621,鼻を鳴らす,ぐす、と<鼻を><鳴らす>敏生に、森はこの少年に感じさせてしまった孤独感に胸が苦しくなった。,1
15237,nonidiom,2621,鼻を鳴らす,ある時には，ハミングするような音を出したり，<鼻を>クンクン<鳴らしたり，また，>ある時には...,1


In [ ]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("tohoku-nlp/bert-base-japanese")

# Tokenize the data
train_encodings = tokenizer(list(df_train['submission']), truncation=True, padding=True, return_tensors="pt")
val_encodings = tokenizer(list(df_eval['submission']), truncation=True, padding=True, return_tensors="pt")
test_encodings = tokenizer(list(df_test['submission']), truncation=True, padding=True, return_tensors="pt")
# Convert to torch tensors
train_labels = torch.tensor(df_train['category_encoded'].values)
val_labels = torch.tensor(df_eval['category_encoded'].values)
test_labels = torch.tensor(df_test['category_encoded'].values)

# Prepare to datasets
train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)
test_dataset = TextDataset(test_encodings, test_labels)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
num_labels = len(label_encoder.classes_)
model = AutoModelForSequenceClassification.from_pretrained("tohoku-nlp/bert-base-japanese", num_labels=num_labels)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-6)

epochs = 4
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Average loss across all batches
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs} | Train Loss: {avg_train_loss}")

    # Evaluation step
    model.eval()
    total_eval_accuracy = 0
    for batch in tqdm(val_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        accuracy = (predictions == batch['labels']).cpu().numpy().mean() * 100
        total_eval_accuracy += accuracy

    avg_val_accuracy = total_eval_accuracy / len(val_loader)
    print(f"Epoch {epoch + 1}/{epochs} | Validation Accuracy: {avg_val_accuracy:.2f}%")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at tohoku-nlp/bert-base-japanese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/376 [00:00<?, ?it/s]<ipython-input-121-845b27163e62>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-121-845b27163e62>:8: U

Epoch 1/4 | Train Loss: 0.252879755873669


100%|██████████| 94/94 [00:01<00:00, 49.37it/s]


Epoch 1/4 | Validation Accuracy: 95.21%


100%|██████████| 376/376 [00:33<00:00, 11.30it/s]


...

Epoch 3/4 | Validation Accuracy: 96.68%


100%|██████████| 376/376 [00:33<00:00, 11.22it/s]


Epoch 4/4 | Train Loss: 0.009464144940330757


100%|██████████| 94/94 [00:01<00:00, 49.45it/s]

Epoch 4/4 | Validation Accuracy: 97.87%


In [ ]:
# Ensure the model is in evaluation mode
model.eval()

# Move the model to the appropriate device
model.to(device)

# Initialize variables to track accuracy
total_correct = 0
total_predictions = 0

# Store true labels and predictions for F1 score calculation
all_labels = []
all_predictions = []

# No gradient updates needed for evaluation
with torch.no_grad():
    for batch in test_loader:
        # Move batch to the same device as the model
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)

        # Get predictions
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        # Update tracking variables
        total_correct += (predictions == batch['labels']).sum().item()
        total_predictions += predictions.size(0)

        # Store predictions and true labels
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(batch['labels'].cpu().numpy())

# Calculate accuracy
test_accuracy = total_correct / total_predictions
print(f'Test Accuracy: {test_accuracy:.4f}')

# Calculate F1 scores
f1_weighted = f1_score(all_labels, all_predictions, average='weighted')
f1_macro = f1_score(all_labels, all_predictions, average='macro')

print(f'Weighted F1 Score: {f1_weighted:.4f}')
print(f'Macro F1 Score: {f1_macro:.4f}')


<ipython-input-121-845b27163e62>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-121-845b27163e62>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Test Accuracy: 0.8447
Weighted F1 Score: 0.8456
Macro F1 Score: 0.8146


In [ ]:
# Generate classification report
report = classification_report(all_labels, all_predictions, digits=4)
print(f'Classification Report for {modelname}, language {language.upper()}, Train {input.upper()}, Test {test.upper()}:')
print(report)

Classification Report for JapaneseBERT, language JP, Train GPT4, Test OPENMWE:
              precision    recall  f1-score   support

           0     0.8967    0.8821    0.8894     10781
           1     0.7257    0.7544    0.7398      4458

    accuracy                         0.8447     15239
   macro avg     0.8112    0.8182    0.8146     15239
weighted avg     0.8467    0.8447    0.8456     15239

